In [1]:
import cv2
import numpy as np
import pandas as pd
from skimage import filters
from skimage.filters import gabor_kernel
from scipy.ndimage import convolve
import matplotlib.pyplot as plt 
import os
from tqdm import tqdm

In [2]:
def create_gabor_filter(size, u0, v0, delta_x, delta_y):
    #size: kernel size
    #u0, v0: spatial frequency points
    #delta_x, delta_y: spatial scales

    y, x = np.mgrid[-size//2:size//2, -size//2:size//2]
    
    gaussian = np.exp(-0.5 * (x**2/delta_x**2 + y**2/delta_y**2))

    sinusoid = np.exp(-2j * np.pi * (u0*x + v0*y))
    
    return gaussian * sinusoid


In [3]:
def compute_gist_descriptor(image_path):

    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    if image is None:
        return None
        
    image = cv2.resize(image, (128, 128))
    image = image.astype(np.float32) / 255.0
    
    local_mean = cv2.GaussianBlur(image, (5, 5), 1.0)
    local_std = np.sqrt(cv2.GaussianBlur(image**2, (5, 5), 1.0) - local_mean**2)
    image = (image - local_mean) / (local_std + 1e-8)
    
    features = []
    scales = [2, 4, 8, 16]
    orientations = 8
    
    for delta in scales:
        for theta in range(orientations):
            angle = theta * np.pi / orientations
            u0 = np.cos(angle) / delta
            v0 = np.sin(angle) / delta
            
            gabor_filter = create_gabor_filter(size=31,u0=u0,v0=v0,delta_x=delta,delta_y=delta)
            
            filtered = cv2.filter2D(image, cv2.CV_32F, np.real(gabor_filter))
            
            block_h = image.shape[0] // 4
            block_w = image.shape[1] // 4
            
            for i in range(4):
                for j in range(4):
                    block = filtered[i*block_h:(i+1)*block_h, j*block_w:(j+1)*block_w]
                    energy = np.mean(np.abs(block))
                    features.append(energy)
    
    return np.array(features)


In [18]:
def process_directory(base_path):
    all_features = []
    all_labels = []
    image_paths = []

    for root, dirs, files in os.walk(base_path):
        for file in files:
            if file.endswith(('.png', '.jpg', '.jpeg')):
                label = 0 if 'Negative' in root else 1
                image_paths.append((os.path.join(root, file), label))
    
    for image_path, label in tqdm(image_paths, desc="Processing"):
        features = compute_gist_descriptor(image_path)
        if features is not None:
            all_features.append(features)
            all_labels.append(label)
    
    df = pd.DataFrame(all_features)
    df['label'] = all_labels
    
    return df

In [34]:
base_path = r'D:\Rice_photos\Rice_photos\Xi23'
df = process_directory(base_path)


Processing:   0%|          | 0/4145 [00:00<?, ?it/s]

C:\Users\SuongLe\AppData\Local\Temp\ipykernel_20924\3483533326.py:11: RuntimeWarning: invalid value encountered in sqrt
  local_std = np.sqrt(cv2.GaussianBlur(image**2, (5, 5), 1.0) - local_mean**2)
Processing: 100%|██████████| 4145/4145 [01:26<00:00, 47.74it/s]


In [33]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2006 entries, 0 to 2005
Columns: 513 entries, 0 to label
dtypes: float32(512), int64(1)
memory usage: 3.9 MB


In [35]:
df.to_csv('gist_features_Xi23.csv', index=False)